In [ ]:
import os , shutil , glob ,cv2
import geojson
import numpy as np
import json 
from xml.dom.minidom import parseString
from lxml.etree import Element, SubElement, tostring
from os.path import join
import pandas as pd
import shutil
from csv import reader

In [ ]:
# Converts TEXT Into XML yolo/text(class xmin ymin xmax ymax)
def xml_transform(root):
    # Text Files Dir Name
    class_path  = join(root, 'Stirling_Yolo')
    ids = list()
    l=os.listdir(class_path)

    check = '.DS_Store' in l
    if check == True:
        l.remove('.DS_Store')

    ids=[x.split('.')[0] for x in l]
    # Text Files Dir Name
    annopath = join(root, 'Stirling_Yolo', '%s.txt')
    # Images Dir Name   and Images Extention
    imgpath = join(root, "Stirling_imgs", '%s.jpg')
    # Output path dir Name
    outpath = join(root, "stirling_xml", '%s.xml')
    ignored=[]
    for i in range(len(ids)):
        img_id = ids[i]
        
        try:
            img= cv2.imread(imgpath % img_id)
            height, width, channels = img.shape 
            # Images Extention
            img_name = img_id + '.jpg'
            node_root = Element('annotation')
            node_folder = SubElement(node_root, 'folder')
            node_folder.text = img_name

            node_filename = SubElement(node_root, 'filename')
            node_filename.text = img_name

            node_source= SubElement(node_root, 'source')
            node_database = SubElement(node_source, 'database')
            node_database.text = 'SLD database'

            node_size = SubElement(node_root, 'size')
            node_width = SubElement(node_size, 'width')
            node_width.text = str(width)

            node_height = SubElement(node_size, 'height')
            node_height.text = str(height)

            node_depth = SubElement(node_size, 'depth')
            node_depth.text = str(channels)

            node_segmented = SubElement(node_root, 'detections')
            node_segmented.text = '0'

            target = (annopath % img_id) # 
            if os.path.exists(target):
                f=open(target,'r')
                label_norm= f.readlines()     
                for i in range(len(label_norm)):
        
                    labels_conv = label_norm[i].split(" ")
                    new_label = [0] * 5

                    # Convert Yolo Format to Pascal VOC format
                    box_width = float(labels_conv[3]) * width
                    box_height = float(labels_conv[4]) * height
                    new_label[1] = str(int(float(labels_conv[1]) * width - (box_width / 2)))
                    new_label[2] = str(int(float(labels_conv[2]) * height - (box_height / 2)))
                    new_label[3] = str(int(float(labels_conv[1]) * width + (box_width / 2)))
                    new_label[4] = str(int(float(labels_conv[2]) * height + (box_height / 2)))

                    node_object = SubElement(node_root, 'object')
                    node_name = SubElement(node_object, 'name')
                    # Class Name in XML file
                    '''
                    Classes:{0:'car', 1:'motorcycle', 2:'bus',
                     3:'truck'}
                    '''
                    if labels_conv[0]=='0':
                        cat='car'
                    elif labels_conv[0]=='1':
                        cat='motorcycle'
                    elif labels_conv[0]=='2':
                        cat='bus'
                    elif labels_conv[0]=='3':
                        cat='truck'
                        
                    node_name.text = cat

                    node_pose = SubElement(node_object, 'pose')
                    node_pose.text = 'Unspecified'

                    # Add bounding boxes subelements
                    node_truncated = SubElement(node_object, 'truncated')
                    node_truncated.text = '0'
                    node_difficult = SubElement(node_object, 'difficult')
                    node_difficult.text = '0'
                    node_bndbox = SubElement(node_object, 'bndbox')
                    node_xmin = SubElement(node_bndbox, 'xmin')
                    node_xmin.text = str(new_label[1])
                    node_ymin = SubElement(node_bndbox, 'ymin')
                    node_ymin.text = str(new_label[2])
                    node_xmax = SubElement(node_bndbox, 'xmax')
                    node_xmax.text =  str(new_label[3])
                    node_ymax = SubElement(node_bndbox, 'ymax')
                    node_ymax.text = str(new_label[4])
                    xml = tostring(node_root, pretty_print=True)
            f =  open(outpath % img_id, "wb")
            f.write(xml)
            f.close()
        except Exception as e:
            print(e)
            print(img_id)
            ignored.append(img_id+" \n")
            continue

In [ ]:
root="./yolo_files/" 
xml_transform(root)